## Backtesting Sesion 1
### Evaluación de Señales por Bootstrapping 

Este cuaderno muestra un ejemplo de como realizar una evaluación fuera de los datos de optimización, utilizando muestreo aleatorio.  Las ideas generales las podemos resumir en:
- Escoger un conjunto de parámetros a partir de la exploración sobre un periodo de tiempo puede llevar a un sobre-ajuste de la estrategia
- Hacer una validación de ventanas deslizantes hacia adelante presenta el inconveniente de que solo se evalua sobre un único camino posible durante el periodo disponible
- Una alternativa es hacer muestreo aleatorio para determinar los periodos de ajuste y de prueba y analizar las distribuciones resultantes de rendimiento.


____

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import seaborn as sns

### Datos 
Asumimos que tenemos un dataset disponible con datos de mercado.
Partimos de un snapshot de acciones del IBEX35

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Generamos un único dataframe con los precios de cierre

In [ ]:
close_dict = {tk: df.close for tk,df in stock_data.items()}
stock_close = pd.DataFrame(close_dict)
stock_close.head()

Trabajaremos primero con un solo valor, que podemos ir variando

In [ ]:
ticker = 'TEF'
stock_series = stock_close[ticker].dropna()

In [ ]:
stock_series.plot()

___

In [ ]:
from stoosc import Sto # clase con las funciones del oscilador estocastico

In [ ]:
Sto.backtest_so_returns(stock_series, win=25, obought=0.8, osold=0.2)

Función para evaluar las diferentes combinaciones (igual que en 1.3)

In [ ]:
obought_params = [0.70, 0.80, 0.90]
osold_params = [0.10, 0.20, 0.30]
win_params = [20, 30, 50]
combined_params = list(itertools.product(win_params, obought_params, osold_params))
print(len(combined_params))
combined_params[:10]

In [ ]:
def explore_sto_params(params_product, vseries):
    result = {}
    for iparams in params_product:
        (w, b, s) = iparams
        result[iparams] = Sto.backtest_so_returns(vseries, win=w, obought=b, osold=s)
    rseries = pd.Series(result)
    return rseries.idxmax(), rseries.max()

____
### Bootstrapping Validation
La idea consiste en tomar ventanas temporales aleatorias dentro del rango de fechas para:
    - no hacer una validación que tenga fechas fijas (inicio o fin de año)
    - tengamos más repeticiones que nos permitan explorar más 

In [ ]:
window_size = 500  # tamaño de la ventana de evaluacion
n_samples = 20
n_all_days = stock_series.shape[0]
available_size = n_all_days - window_size

start_points = np.random.randint(0, available_size, n_samples)
start_points

In [ ]:
window_samples = [stock_series.index[i: i+window_size] for i in start_points]
window_samples[:3]

___

Para visualizar creamos una gráfica que nos muestre la distribución de los periodos muestreados

In [ ]:
def plot_bootstrap_window_sample(window_samples):
    fig, ax = plt.subplots(figsize=(12,5))
    n = len(window_samples)
    for i, isample in enumerate(window_samples):
        istart, iend = isample[0], isample[-1]
        hi, hi_next = i/n, (i+0.9)/n
        ax.axvspan(istart, iend, hi, hi_next, alpha=0.2)

In [ ]:
plot_bootstrap_window_sample(window_samples)

Ahora utilizamos la forma de generar las ventanas aleatorias para hacer una evaluación
sobre cada período de ajuste

In [ ]:
def bootstrap_eval(params_product, vseries, window, n):
    n_all_days = vseries.shape[0]
    available_size = n_all_days - window
    start_points = np.random.randint(0, available_size, n)
    
    window_samples = [stock_series.index[i: i+window_size] for i in start_points]
    result = []
    for i, isample in enumerate(window_samples):
        sample_serie = vseries.loc[isample]
        params, val_max = explore_sto_params(params_product, sample_serie)
        print(i, params, val_max)
        ires = {
            'start': isample[0],
            'end': isample[-1],
            'params': params,
            'best_ret': val_max,
        }
        result.append(ires)
                      
    return result

In [ ]:
bootres = bootstrap_eval(combined_params, stock_series, 500, 15)

____
### Bootstrapping con Test
Podemos aplicar este mismo enfoque, pero estimando el resultado, fuera del periodo de ajuste.
- Necesitamos generar para cada muestra de ajuste, una ventana alternativa que no solape
- Los parámetros seleccionados en el ajuste lo evaluamos sobre esta nueva ventana  

In [ ]:
window_samples[0].max()

In [ ]:
def interval_overlaps(s1, s2):
    """Determina si dos intervalos s1=(a1, b1) y s2=(a2, b2)  solapan, 
    verificando el menor punto final y el mayor punto inicial
    """
    check = min(s1[-1], s2[-1]) - max(s1[0], s2[0])
    return check > 0  

In [ ]:
interval_overlaps([49, 149], [100, 200])

In [ ]:
interval_overlaps([49, 149], [200, 300])

____

Podemos para un 'intervalo' de indices, pedir generar otro hasta que no se solapen 

In [ ]:
available_size

In [ ]:
fitstart = np.random.randint(0, available_size)
fit_inverval = (fitstart, fitstart + window_size)
fit_inverval

In [ ]:
def gen_interval(win, bound=available_size):
    vstart = np.random.randint(0, bound)
    return vstart, vstart + win 

In [ ]:
gen_interval(300)

In [ ]:
gen_interval(500)

In [ ]:
test_interval = gen_interval(750)
print(test_interval)
while interval_overlaps(fit_inverval, test_interval):
    test_interval = gen_interval(750)
    print(test_interval)

In [ ]:
def bootstrap_fit_test_samples(vseries, num_samples, fit_size, test_size):
    """
    Función que genera un muestreo de ventandas de ajuste y de test
    """
    n_all_days = stock_series.shape[0]
    available_size = n_all_days - window_size
    
    samples = []  # (fit_date_index, test_date_index)
    for i in range(num_samples):
        fit_idxs = gen_interval(fit_size, bound=available_size)
        test_idxs = gen_interval(test_size, bound=available_size)

        while interval_overlaps(fit_idxs, test_idxs):
            test_idxs = gen_interval(test_size, bound=available_size)
        samples.append(
            (vseries.index[fit_idxs[0]: fit_idxs[1]],
             vseries.index[test_idxs[0]: test_idxs[1]])
        )
    return samples 

In [ ]:
fittest_samples = bootstrap_fit_test_samples(stock_series, 15, 500, 750)

In [ ]:
fittest_samples[:2]

____

Modificamos la función de graficar las ventanas muestreadas

In [ ]:
def plot_bootstrap_fittest_samples(samples):
    fig, ax = plt.subplots(figsize=(12,5))
    n = len(samples)
    for i, isample in enumerate(samples):
        fitstart, fitend = isample[0][0], isample[0][-1]
        hi, hi_next = i/n, (i+0.9)/n
        ax.axvspan(fitstart, fitend, hi, hi_next, facecolor='blue', alpha=0.2)
        teststart, testend = isample[1][0], isample[1][-1]
        ax.axvspan(teststart, testend, hi, hi_next, facecolor='green', alpha=0.2)
        
        

In [ ]:
plot_bootstrap_fittest_samples(fittest_samples)

Finalmente modificamos la función de evaluación bootstrapping para dar la estimación de rendimiento
sobre la ventana de test

In [ ]:
def bootstrap_eval_test(params_product, vseries, fit_win, test_win, n):
    
    fittest_samples = bootstrap_fit_test_samples(vseries, n, fit_win, test_win)
    result = []
    for i, (fitsample, testsample) in enumerate(fittest_samples):
        fit_serie = vseries.loc[fitsample]
        test_serie = vseries.loc[testsample]
        
        params, val_max = explore_sto_params(params_product, fit_serie)
        w, b, s = params
        
        test_ret = Sto.backtest_so_returns(test_serie, win=w, obought=b, osold=s)
  
        print(i, params, val_max, test_ret)
        ires = {
            'start': fit_serie[0],
            'end': fit_serie[-1],
            'params': params,
            'best_fit_ret': val_max,
            'test_ret': test_ret
        }
        result.append(ires)
                      
    return result

In [ ]:
bootres = bootstrap_eval_test(combined_params, stock_series, 
                              fit_win=500, 
                              test_win=750,
                              n=50)

In [ ]:
show_rets = [r['test_ret'] for r in bootres]
plt.hist(show_rets)

### Observaciones Finales
- El muestreo no es independiente, por la limitación del solapamiento en los extremos
- Lo relevante de este esquema es la evaluación fuera de la ventana de ajuste. Aquí podemos plantear alternativas, 
   por ejemplo elegir la combinación de parámetros que más se repita entre los mejores rendimientos

____